<a href="https://colab.research.google.com/github/Yashasvi1515/GrammarCorrection/blob/main/GrammarCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
print(os.listdir())


['.config', 'sample_data']


In [2]:
import torch
torch.cuda.is_available()  # Should return True
!nvidia-smi

Tue Oct  7 10:59:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install torch

In [4]:
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA available: True
GPU count: 1
GPU name: Tesla T4


In [5]:
!pip install transformers datasets torch


In [6]:
!pip install sentencepiece -q

In [7]:
pip install nltk


In [8]:
from datasets import load_dataset, DatasetDict

# 1. Load the grammar correction dataset from Hugging Face
ds = load_dataset("agentlans/grammar-correction")

# 2. Create train/validation split from train data
train_valid_split = ds["train"].train_test_split(test_size=0.5, seed=42)

# 3. Build a DatasetDict with train, validation, and test
dataset = DatasetDict({
    "train": train_valid_split["train"],
    "validation": train_valid_split["test"],
    "test": ds["validation"]   # use official validation as test
})

print(dataset)
print(dataset["train"][0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl.zst:   0%|          | 0.00/6.91M [00:00<?, ?B/s]

validation.jsonl.zst:   0%|          | 0.00/1.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 25000
    })
})
{'input': 'Copyright 2019 Versant Law | The San Francisco Business and Real Estate Law Firm; at All rights reserved.', 'output': 'Copyright 2019 Versant Law | The San Francisco Business and Real Estate Law Firm. All rights reserved.'}


In [9]:
!pip install transformers

In [10]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 87.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

# -----------------------------
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# -----------------------------
# -----------------------------
def preprocess(examples):
    # Add prefix for grammar correction task
    inputs = ["fix grammar: " + s for s in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Tokenize targets
    labels = tokenizer(examples["output"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_datasets = {}
for split in dataset:  # dataset should have train/validation/test splits
    tokenized_datasets[split] = dataset[split].map(preprocess, batched=True)

# -----------------------------
# -----------------------------
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# -----------------------------
# -----------------------------
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

# -----------------------------
# -----------------------------
trainer.train()

# -----------------------------
# -----------------------------
trainer.save_model("./grammar_correction_model")

# -----------------------------
# -----------------------------
tokenizer = T5Tokenizer.from_pretrained("./grammar_correction_model")
model = T5ForConditionalGeneration.from_pretrained("./grammar_correction_model")

# -----------------------------
# -----------------------------
def correct_sentence(sentence):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    input_text = "fix grammar: " + sentence
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    output_ids = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    corrected = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    if corrected.strip() == "":
        corrected = "Model did not generate output."

    return corrected


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yashasvishukla8597 (yashasvishukla8597-indian-institute-of-technology-indore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.226400,0.207621
2,0.220000,0.199956
3,0.215000,0.197986
4,0.207400,0.196932


Epoch,Training Loss,Validation Loss
1,0.226400,0.207621
2,0.220000,0.199956
3,0.215000,0.197986
4,0.207400,0.196932
5,0.211500,0.196530


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


In [12]:
sentence = "He is very shy for hide the truth"
corrected = correct_sentence(sentence)
print("Input:", sentence)
print("Corrected:", corrected)

Input: He is very shy for hide the truth
Corrected: He is very shy to hide the truth.


In [13]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [14]:
import evaluate
import numpy as np
bleu = evaluate.load("bleu")
accuracy = evaluate.load("accuracy")


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode token IDs back to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip whitespace
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Compute BLEU score
    bleu_score = bleu.compute(predictions=decoded_preds, references=[[lbl] for lbl in decoded_labels])

    # Compute exact match accuracy
    acc_score = accuracy.compute(
        references=decoded_labels,
        predictions=decoded_preds
    )

    return {
        "bleu": bleu_score["bleu"],
        "accuracy": acc_score["accuracy"]
    }
